In [36]:
import os
import fitz  
import shutil
import json
from pdf2image import convert_from_path
import glob
from docx import Document
import pandas as pd




In [3]:
def extract_text_annotations(response):
    # 初始化一個空的列表，用於存儲提取出的文本註釋
    text_annotations = []
    
    # 遍歷回應中的每個文本註釋
    for annotation in response.text_annotations:
        # 將每個註釋的描述（即文本內容）添加到列表中
        text_annotations.append(annotation.description)
    
    # 返回包含所有提取出的文本註釋的列表
    return text_annotations


In [4]:
def detect_text(path):
    """Detects text in the file."""
    
    # 從 Google Cloud Vision 模組中導入 vision
    from google.cloud import vision

    # 創建一個 ImageAnnotatorClient 客戶端對象
    client = vision.ImageAnnotatorClient()

    # 以二進制模式打開指定路徑的圖像文件
    with open(path, "rb") as image_file:
        content = image_file.read()  # 讀取圖像文件內容

    # 使用讀取的圖像內容創建一個 vision.Image 對象
    image = vision.Image(content=content)

    # 調用客戶端的 text_detection 方法檢測圖像中的文本
    response = client.text_detection(image=image)
    
    # 提取檢測到的文本註釋
    texts = extract_text_annotations(response)
    
    # 初始化一個空的字符串，用於存儲所有檢測到的文本
    returnvalue = ""
    
    # 遍歷提取出的每個文本
    for text in texts:
        returnvalue += text  # 將文本添加到返回值字符串中
        returnvalue += " "  # 在每個文本後面加一個空格分隔

    # 檢查回應中是否有錯誤信息
    if response.error.message:
        # 如果有錯誤信息，拋出異常並顯示錯誤信息和幫助鏈接
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
        )
    
    # 返回包含所有檢測到文本的字符串
    return returnvalue


In [5]:
def pdf_to_jpg(pdf_path, output_path, quality=100):
    # 從 pdf2image 模組中導入 convert_from_path 函數
    from pdf2image import convert_from_path
    
    # 將 PDF 文件轉換為圖像，每頁生成一個圖像對象
    images = convert_from_path(pdf_path, 800)
    
    # 初始化一個空列表，用於存儲生成的圖像文件路徑
    image_paths = []
    
    # 遍歷所有生成的圖像對象
    for i, image in enumerate(images):
        # 構建每個圖像文件的保存路徑，包含頁碼
        image_path = f"{output_path}_page_{i+1}.jpg"
        
        # 將圖像保存為 JPEG 格式文件，指定質量參數
        image.save(image_path, "JPEG", quality=quality)
        
        # 打印轉換成功信息
        print(f"{pdf_path} converted to {image_path}")
        
        # 將圖像文件路徑添加到列表中
        image_paths.append(image_path)
    
    # 返回包含所有生成圖像文件路徑的列表
    return image_paths


In [6]:
import os
import fitz  # PyMuPDF

def pdf_to_text(pdf_path, text_path):
    # 如果 PDF 文件大小為 0，則跳過處理並打印提示信息
    if os.path.getsize(pdf_path) == 0:
        print(f"Skipping empty file: {pdf_path}")
        return

    # 打開 PDF 文件
    doc = fitz.open(pdf_path)
    text = ""

    # 遍歷 PDF 文件的每一頁
    for page_num in range(doc.page_count):
        page = doc[page_num]
        # 提取頁面的文本內容
        text += page.get_text()

    # 如果提取的文本內容為空，則假設 PDF 文件是圖片格式
    if text == "":
        print(f"{pdf_path} is image")
        print(text_path)
        
        # 將文本文件路徑拆分為文件名和擴展名
        words = text_path.split('.')
        
        # 將 PDF 轉換為圖像文件
        image_paths = pdf_to_jpg(pdf_path, words[0])
        
        # 對每個圖像文件進行文本檢測並累加檢測到的文本
        for image_path in image_paths:
            text += detect_text(image_path)

    # 將提取或檢測到的文本寫入指定的文本文件
    with open(text_path, "w", encoding="utf-8") as text_file:
        text_file.write(text)

    # 關閉 PDF 文件對象
    doc.close()


In [19]:
from docx import Document

def docx_to_txt(docx_file, txt_file):
    # 打開 DOCX 文件
    doc = Document(docx_file)
    
    # 以寫模式打開目標 TXT 文件，並設置編碼為 UTF-8
    with open(txt_file, 'w', encoding='utf-8') as f:
        # 遍歷 DOCX 文件中的每個段落
        for paragraph in doc.paragraphs:
            # 將段落的文本內容寫入 TXT 文件，並在每個段落後添加換行符
            f.write(paragraph.text + '\n')


In [20]:
import os

def batch_convert_pdfs(input_folder, output_folder):
    # 檢查輸出文件夾是否存在，如果不存在則創建
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # 初始化計數器，用於計算處理的 PDF 和 DOCX 文件數量
    pdf_count = 0
    doc_count = 0
    
    # 遍歷輸入文件夾及其子文件夾中的所有文件
    for root, dirs, files in os.walk(input_folder):
        for filename in files:
            # 獲取文件的完整路徑
            pdf_path = os.path.join(root, filename)
            
            # 獲取文件相對於輸入文件夾的相對路徑
            relative_path = os.path.relpath(pdf_path, input_folder)
            
            # 構建對應的文本文件名和路徑
            txt_filename = os.path.splitext(relative_path)[0] + ".txt"
            txt_path = os.path.join(output_folder, txt_filename)
            
            # 確保目標文件夾存在
            os.makedirs(os.path.dirname(txt_path), exist_ok=True)
            
            # 判斷文件類型並進行相應處理
            if filename.endswith(".pdf"):
                pdf_count += 1
                pdf_to_text(pdf_path, txt_path)
            elif filename.endswith(".docx"):
                doc_count += 1
                docx_to_txt(pdf_path, txt_path)

    # 打印處理的 PDF 和 DOCX 文件數量
    print(pdf_count)
    print(doc_count)


In [8]:
import os
import shutil
import json

def copy_txt_files(src_folder, dest_folder):
    # 初始化兩個字典，用於記錄文件的日期和類型信息
    date = {}
    type = {}
    
    # 檢查目標文件夾是否存在，如果不存在則創建該文件夾
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)

    # 初始化索引，用於命名複製的文本文件
    index = 0
    
    # 遍歷源文件夾及其子文件夾中的所有文件
    for root, dirs, files in os.walk(src_folder):
        for txt_file in files:
            # 檢查文件是否為文本文件
            if txt_file.endswith('.txt'):
                # 獲取源文件的完整路徑
                src_path = os.path.join(root, txt_file)
                
                # 將源文件的路徑拆分為不同的部分
                r = src_path.split(os.sep)
                
                # 根據文件的第三部分（假設為日期）更新日期字典
                if r[2] in date:
                    date[r[2]].append(index)
                else:
                    date[r[2]] = [index]
                
                # 根據文件的第二部分（假設為類型）更新類型字典
                if r[1] in type:
                    type[r[1]].append(index)
                else:
                    type[r[1]] = [index]
                
                # 獲取相對於源文件夾的文件路徑
                relative_path = os.path.relpath(src_path, src_folder)
                
                # 構建目標文件的路徑，使用索引作為文件名
                dest_path = os.path.join(dest_folder, f'{index}.txt')
                
                # 複製文件到目標文件夾，保留原文件的元數據
                shutil.copy2(src_path, dest_path)
                
                # 增加索引
                index += 1

    # 將日期和類型字典寫入到文件中，保存在當前目錄下
    with open('date.txt', 'w') as file:
        json.dump(date, file)
    with open('type.txt', 'w') as file:
        json.dump(type, file)


In [9]:
import os
import glob

def delete_jpg_files(folder_path):
    # 定義要刪除的文件擴展名
    file_extension = '.jpg'
    
    # 使用 glob 模組獲取指定文件夾中所有以 .jpg 結尾的文件列表
    file_list = glob.glob(os.path.join(folder_path, f'*{file_extension}'))
    
    # 遍歷文件列表，刪除每個文件
    for file_path in file_list:
        try:
            os.remove(file_path)  # 刪除文件
            print(f'{file_path} 已删除')
        except Exception as e:
            print(f'删除 {file_path} 时出错: {e}')
    
    # 獲取文件夾中的所有子文件夾
    subfolders = [f.path for f in os.scandir(folder_path) if f.is_dir()]
    
    # 遍歷每個子文件夾，遞歸地調用 delete_jpg_files 函數
    for subfolder in subfolders:
        delete_jpg_files(subfolder)


In [28]:
input_folder = "Newspaper article project/News article"
output_folder = "data"

batch_convert_pdfs(input_folder, output_folder)

Newspaper article project/News article\Boston Globe\202104\Boston Globe_04172021.pdf is image
data\Boston Globe\202104\Boston Globe_04172021.txt
Newspaper article project/News article\Boston Globe\202104\Boston Globe_04172021.pdf converted to data\Boston Globe\202104\Boston Globe_04172021_page_1.jpg
Newspaper article project/News article\Boston Globe\202104\Boston Globe_04172021.pdf converted to data\Boston Globe\202104\Boston Globe_04172021_page_2.jpg
Newspaper article project/News article\Boston Globe\202104\Boston Globe_04172021.pdf converted to data\Boston Globe\202104\Boston Globe_04172021_page_3.jpg
Newspaper article project/News article\Boston Globe\202104\Boston Globe_04182021.pdf is image
data\Boston Globe\202104\Boston Globe_04182021.txt
Newspaper article project/News article\Boston Globe\202104\Boston Globe_04182021.pdf converted to data\Boston Globe\202104\Boston Globe_04182021_page_1.jpg
Newspaper article project/News article\Boston Globe\202104\Boston Globe_04182021.pdf c

In [31]:
delete_jpg_files("data")

data\Boston Globe\202104\Boston Globe_04172021_page_1.jpg 已删除
data\Boston Globe\202104\Boston Globe_04172021_page_2.jpg 已删除
data\Boston Globe\202104\Boston Globe_04172021_page_3.jpg 已删除
data\Boston Globe\202104\Boston Globe_04182021_page_1.jpg 已删除
data\Boston Globe\202104\Boston Globe_04182021_page_2.jpg 已删除
data\Boston Globe\202104\Boston Globe_04182021_page_3.jpg 已删除
data\Boston Globe\202107\Boston Globe_07142021_page_1.jpg 已删除
data\Boston Globe\202107\Boston Globe_07142021_page_2.jpg 已删除
data\Chicago Tribune\202104\Chicago Tribune_04192021_page_1.jpg 已删除
data\Chicago Tribune\202104\Chicago Tribune_04192021_page_2.jpg 已删除
data\LA Times\202104\LA Times_04162021_page_1.jpg 已删除
data\LA Times\202104\LA Times_04162021_page_2.jpg 已删除
data\LA Times\202104\LA Times_04162021_page_3.jpg 已删除
data\LA Times\202104\LA Times_04182021_page_1.jpg 已删除
data\LA Times\202104\LA Times_04182021_page_2.jpg 已删除
data\LA Times\202105\LA Times_05052021_page_1.jpg 已删除
data\LA Times\202105\LA Times_05052021_page_

In [37]:
from pathlib import Path

# 要遍歷的目錄路徑
folder_path = Path('data')
dest_folder = "txt"
if not os.path.exists(dest_folder):
    os.makedirs(dest_folder)
index = 0
labels = []

# 使用 Path.glob 遍歷目錄
for file_path in folder_path.glob('**/*'):
    if file_path.is_file():
        s = str(file_path).split('\\')
        label = {"id": index, "source": s[1], "date": s[2]}
        print(s)
        print(label["id"])
        labels.append(label)
        dest_path = os.path.join(dest_folder, f'{index}.txt')
        shutil.copy2(file_path, dest_path)  
        index += 1
df = pd.DataFrame(labels)
df.to_csv("txt/labels.csv", index=False)



['data', 'Boston Globe', '202004', 'Boston Globe_04142020.txt']
0
['data', 'Boston Globe', '202004', 'Boston Globe_04162020.txt']
1
['data', 'Boston Globe', '202009', 'Boston Globe_09292020.txt']
2
['data', 'Boston Globe', '202102', 'Boston Globe_02132021.txt']
3
['data', 'Boston Globe', '202103', 'Boston Globe_03042021.txt']
4
['data', 'Boston Globe', '202103', 'Boston Globe_03172021(2).txt']
5
['data', 'Boston Globe', '202103', 'Boston Globe_03172021(3).txt']
6
['data', 'Boston Globe', '202103', 'Boston Globe_03172021(4).txt']
7
['data', 'Boston Globe', '202103', 'Boston Globe_03172021(5).txt']
8
['data', 'Boston Globe', '202103', 'Boston Globe_03172021.txt']
9
['data', 'Boston Globe', '202103', 'Boston Globe_03182021(2).txt']
10
['data', 'Boston Globe', '202103', 'Boston Globe_03182021.txt']
11
['data', 'Boston Globe', '202103', 'Boston Globe_03192021.txt']
12
['data', 'Boston Globe', '202103', 'Boston Globe_03202021.txt']
13
['data', 'Boston Globe', '202103', 'Boston Globe_03222021